<a href="https://colab.research.google.com/github/NTHU-ML-2023-team19/ADReSSo-Notebooks/blob/main/ADReSSo_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate accelerate

In [2]:
from huggingface_hub import login
from google.colab import userdata


login(userdata.get("HuggingFace_token"))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [1]:
# @title Hyper-parameters
MODEL = "wav2vec2-base_ADReSSo" # @param {type:"string"}
DURATION = 100 # @param {type:"integer"}
BATCH = 1 # @param {type:"integer"}

In [2]:
from datasets import load_dataset, Audio


dataset = load_dataset("nevikw39/ADReSSo", split="test").cast_column(
    "audio", Audio(sampling_rate=16_000)
)

In [3]:
from random import randint


for i in dataset:
    i["audio"]["array"] = i["audio"]["array"][(x := randint(0, len(i["audio"]["array"]) - (y := min(len(i["audio"]["array"]), 16_000*DURATION)))) : x + y]

In [4]:
from transformers import pipeline


pipe = pipeline("audio-classification", model="NTHU-ML-2023-team19/" + MODEL, device=0)

In [5]:
import evaluate


label2int = {"ad": 0, "cn": 1}
accuracy = evaluate.load("accuracy")
accuracy.compute(predictions=list(label2int[i[0]["label"]] for i in pipe(dataset["audio"], batch_size=BATCH)), references=dataset["label"])

{'accuracy': 0.6056338028169014}